In [2]:
import json
import pandas as pd
import requests
import io
from urllib.request import urlopen
import json
import logging
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pandas.io.json import json_normalize

In [10]:
# load JSON
def load_data(filename):
    df = pd.read_json(filename)
    data_sorted = df['formAnswers'].apply(pd.Series).join(df[['swymerId', 'workDurationInSeconds']])
    return data_sorted.fillna("")

In [4]:
from google.colab import files

uploaded = files.upload()

ModuleNotFoundError: No module named 'google'

In [56]:
#load json files
v1_q = load_data('answers-xai-v1-qualification-12.04.23.json')
v1_m = load_data('answers-xai-v1-main-12.04.23.json')
#v3 = load_data('answers-29.03.2023-v3-explanations-group-1.json')

In [57]:
#confert to pandas data frame 
df_v1_q = pd.DataFrame(v1_q)
df_v1_m = pd.DataFrame(v1_m)

df_v1_q

,information-literacy.identify-misinformation,information-seeking.sources.online-newspaper,expectations.sceptical-about-ai,information-seeking.sources.personal-social-networks,personal-code,information-literacy.efficiently-use-subscribed-and-openaccess,information-seeking.purpose.keep-up-to-date,confirm.0,POINTS,information-seeking.truthfulness.facebook,...,newsitem.12.system-evaluation.indications-useful,newsitem.12.rating-before-xai,information-seeking.sources.interview,newsitem.10.system-evaluation.understand-what-system-does,newsitem.10.system-evaluation.xai-features-useful,newsitem.10.system-evaluation.explanations-comprehensible-and-help-assess,newsitem.10.system-evaluation.classified-correctly,newsitem.10.system-evaluation.indications-useful,swymerId,workDurationInSeconds
0,6,6,5,4,X,5,5,Yes,0,2,...,,,,,,,,,CX_741366675,138
1,3,5,4,2,,3,4,Yes,1,4,...,5,68,2,,,,,,CX_42668919057,305
2,6,6,6,6,X,7,6,Yes,0,6,...,,,,,,,,,CX_43722937894,401
3,6,3,2,3,JAG89,7,7,Yes,1,6,...,7,8,,,,,,,CX_58520310894,606
4,3,4,3,4,,2,4,Yes,0,3,...,,62,4,,,,,,CX_471152374088,275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,5,3,2,4,ptr02,6,6,Yes,2,5,...,6,21,,7,6,6,6,7,CX_861056071436,947
210,4,6,5,5,Mma35,6,5,Yes,0,7,...,,27,4,,,,,,CX_02851791642,188
211,7,3,5,7,tam85,7,7,Yes,2,7,...,7,0,,7,7,7,7,7,CX_49122095778,645
212,5,6,2,6,SMN99,6,7,Yes,2,5,...,7,17,,7,7,7,7,7,CX_64903686257,724


In [58]:
df_v1_m

,news-dashboard-evaluation.assist-decisions,ai-system-evaluation.most-useful-explanation-feature,newsitem.7.system-evaluation.understand-what-system-does,ai-system-evaluation.easy-to-follow-what-system-does,newsitem.11.system-evaluation.classified-correctly,ai-system-evaluation.tend-to-trust-system-even-without-knowledge,news-dashboard-evaluation.useful-assess-news-articles,newsitem.8.system-evaluation.classified-correctly,newsitem.11.system-evaluation.explanations-comprehensible-and-help-assess,newsitem.8.system-evaluation.explanations-comprehensible-and-help-assess,...,newsitem.7.rating-after-xai,ai-system-evaluation.benefit-of-doubt,newsitem.8.rating-before-xai,ai-system-evaluation.system-decides-consistently,tutorial.overview,ai-system-evaluation.everyday-work-helpful,ai-system-evaluation.system-has-functionality-for-work,ai-system-evaluation.why-not-helpful-for-everyday-work,swymerId,workDurationInSeconds
0,7,truthfulness-assessment,7,7,7,7,7,6,6,6,...,90,6,85,6,81,,,,CX_221195070694,543
1,4,truthfulness-assessment,4,4,7,4,6,7,4,7,...,95,6,90,6,66,4,5,,CX_171085388881,1787
2,5,information-about-the-source,5,5,7,1,4,5,6,4,...,90,1,99,1,37,,,,CX_341221661476,676
3,7,truthfulness-assessment,6,7,7,7,6,7,6,7,...,95,6,46,7,37,,,,CX_791160737258,1157
4,7,information-about-the-source,7,7,6,4,5,6,6,6,...,57,5,36,2,,,,,CX_01467716489,1279
5,2,information-about-the-source,7,3,6,2,2,6,7,7,...,90,3,90,2,82,,,,CX_241074146890,946
6,3,information-about-the-source,6,2,7,2,3,6,3,3,...,80,5,40,2,60,,,,CX_471195853693,914
7,6,information-about-the-source,5,5,5,6,6,3,5,3,...,63,5,3,5,18,,,,CX_501175503893,1452
8,6,information-about-the-source,6,6,4,5,5,6,6,6,...,81,6,52,5,,,,,CX_94287616400,766
9,7,information-about-the-source,6,6,5,4,6,2,7,7,...,85,6,25,3,,,,,CX_91238436307,830


In [59]:
# remove columns from df_v1_q which are redundant or not needed for the analysis
df_v1_q = df_v1_q.drop(columns=[
  "METADATA.FEATURE",
  "METADATA.GROUP",
  "confirm.0"
])

# get a list of all column names starting with "tutorial."
tutorial_columns = [col for col in df_v1_q.columns if col.startswith("tutorial.")]
tutorial_columns.append("understand-task")

# remove tutorial columns from both dataframes
df_v1_q = df_v1_q.drop(columns=tutorial_columns)
df_v1_m = df_v1_m.drop(columns=tutorial_columns)

# remove "METADATA.PART" columns from both dataframes
df_v1_q = df_v1_q.drop(columns=["METADATA.PART"])
df_v1_m = df_v1_m.drop(columns=["METADATA.PART"])

In [60]:
# merge df_v1_q and df_v1_m on swymerId, 
# only keep rows where aliasId is in both dataframes (inner join)
# keep all columns from both and suffix them with .qualification and .main (e.g. for POINTS)
df_v1 = pd.merge(df_v1_q, df_v1_m, on='swymerId', how='inner', suffixes=('.qualification', '.main'))

In [63]:
df_v1 = df_v1.rename(columns={
  "swymerId": "METADATA.swymerId",
  "workDurationInSeconds.main": "METADATA.workDurationInSeconds.main",
  "workDurationInSeconds.qualification": "METADATA.workDurationInSeconds.qualification",
  "personal-code": "METADATA.personal-code"
})


In [64]:
# order columns alphabetically
df_v1 = df_v1.reindex(sorted(df_v1.columns), axis=1)

In [65]:
from pprint import pprint

columns = list(df_v1.columns)
pprint(columns)

['METADATA.FEATURE',
 'METADATA.GROUP',
 'METADATA.personal-code',
 'METADATA.swymerId',
 'METADATA.workDurationInSeconds.main',
 'METADATA.workDurationInSeconds.qualification',
 'POINTS.main',
 'POINTS.qualification',
 'ai-system-evaluation.additional-functionality',
 'ai-system-evaluation.benefit-of-doubt',
 'ai-system-evaluation.classification-comprehensible',
 'ai-system-evaluation.criteria-to-judge-reliability',
 'ai-system-evaluation.easy-to-follow-what-system-does',
 'ai-system-evaluation.everyday-work-helpful',
 'ai-system-evaluation.know-what-will-happen-next-time',
 'ai-system-evaluation.most-useful-explanation-feature',
 'ai-system-evaluation.other-information-wish',
 'ai-system-evaluation.rely-on-ai-system',
 'ai-system-evaluation.sceptical-about-ai-system',
 'ai-system-evaluation.system-able-to-classify-news-articles',
 'ai-system-evaluation.system-able-to-detect-fake-news',
 'ai-system-evaluation.system-can-correctly-classify-news-articles',
 'ai-system-evaluation.system-

In [111]:
# determine which columns only contain values which can be parsed to numbers
def is_number(s):
    if s == "":
        return True # empty cells are treated as possible numbers
    try:
        float(s)
        return True
    except ValueError:
        return False

# determine which columns only contain values which can be parsed to integers or are empty
numeric_columns = df_v1.columns[df_v1.applymap(is_number).all()]

# change data type of numeric columns to int
df_v1[numeric_columns] = df_v1[numeric_columns].apply(pd.to_numeric, downcast='integer')

In [113]:
# save to xlsx
df_v1.to_excel("data_cleaned.xlsx")